# Системы Управления Базами Данных

## Финансовый Университет при Правительстве РФ

В данной тетради представлены примеры работы с данными в СУБД SQLite на языке программирования Python и с использованием встренных средств в Jupyter Notebook.

### 1. Работа с данными через Jupyter Notebook

В Jupyter Notebook есть встроенные средства для работы с СУБД - для этого нужно установить расширение sql:

In [ ]:
!conda install -y ipython-sql

Убедившись, что расширение установлено, следует загрузить его в среду Jupyter:

In [ ]:
%load_ext sql

Данное расширение позволяет нам использовать "магическую" команду ```%sql```. Как и любая другая "магия" Jupyter, эта команда, запущенная с одним ```%``` - запускает только одну строку SQL-кода (но можно использовать символ переноса строки ```\```), а ```%%sql``` - позволяет запустить сразу несколько SQL-выражений.

### 1.1 Создание/открытие базы данных

Чтобы открыть базу данных, нужно выполнить "магию" ```%sql```, которой в качестве параметров следует передать наименование/тип СУБД и, в случае SQLite, наименование файла. Сам параметр выглядит как URL (он, собственно, им и является): тип СУБД - это тип взаимодействия, затем двоеточие и два символа ```/```, и после этого - путь к файлу как будто текущая директория - корневая.

```
sqlite:// - "мне нужны данные из СУБД SQLite"
/musicians.db3 - "вот из этого файла"
```

In [ ]:
%sql sqlite:///musicians.db3

### 1.2 Работа с таблицами

Создадим таблицу:

In [ ]:
%%sql sqlite://
CREATE TABLE tbl_musician(
    musID INTEGER PRIMARY KEY AUTOINCREMENT, 
    musFirstName VARCHAR(50) NOT NULL,
    musLastName VARCHAR(50) NOT NULL
);

Добавим данные в таблицу:

In [ ]:
%%sql sqlite://
INSERT INTO tbl_musician (musFirstName, musLastName) VALUES ('John', 'Lennon');
INSERT INTO tbl_musician (musFirstName, musLastName) VALUES ('George', 'Harrison');
INSERT INTO tbl_musician (musFirstName, musLastName) VALUES ('Paul', 'McCartney');
INSERT INTO tbl_musician (musFirstName, musLastName) VALUES ('Ringo', 'Starr');
INSERT INTO tbl_musician (musFirstName, musLastName) VALUES ('Jack', 'White');
INSERT INTO tbl_musician (musFirstName, musLastName) VALUES ('Meg', 'White');
INSERT INTO tbl_musician (musFirstName, musLastName) VALUES ('Robert', 'Plant');
INSERT INTO tbl_musician (musFirstName, musLastName) VALUES ('Jimmy', 'Page');

Выведем данные на экран. Обратите внимание: результат выполнения ```SELECT```-запроса - pandas dataframe.

In [ ]:
sqlres = %sql SELECT * from tbl_musician
sqlres

Добавим признак принадлежности к группе:

In [ ]:
%sql ALTER TABLE tbl_musician ADD COLUMN musBandID INTEGER NULL DEFAULT NULL;

In [ ]:
%%sql sqlite://
CREATE TABLE tbl_band(
    bndID INTEGER PRIMARY KEY AUTOINCREMENT, 
    bndName VARCHAR(50) NOT NULL,
    bndCountry VARCHAR(50) NOT NULL
);
INSERT INTO tbl_band (bndName, bndCountry) VALUES ('The Beatles', 'UK');
INSERT INTO tbl_band (bndName, bndCountry) VALUES ('The White Stripes', 'US');
INSERT INTO tbl_band (bndName, bndCountry) VALUES ('Led Zeppelin', 'UK');

In [ ]:
%sql SELECT * FROM tbl_band

#### ЗАДАНИЕ

Напишите и выполните ```UPDATE``` запросы с использованием команды ```%sql```, которые относят музыкантов к соответствующим группам: 
- Леннон, Харрисон, МакКартни и Старр - Битлз, 
- Джек и Мег Уайт - The White Stripes, 
- Плант и Пейдж - Led Zeppelin. 

In [ ]:
# подсказка: используйте выражение IN :
%sql SELECT * FROM tbl_musician \
WHERE musID IN (1,2,3,4)

In [ ]:
# ваш код здесь


## 2. Работа с SQLite из Python

В Python есть стандарт взаимодействия с СУБД, он работает для всех СУБД (включая MySQL, MSSQL, Oracle и конечно, SQLite). Последовательность действий при чтении из СУБД следующая: 
1. Открыть соединение с СУБД
2. Открыть курсор
3. В рамках открытого курсора выполнить ```SELECT``` запрос
4. Получить результаты в виде списка кортежей через функцию ```cursor.fetchall()``` или перебрать курсор в цикле ```for```

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect('musicians.db3') # соединение
cursor = con.cursor() # курсор

In [ ]:
cursor.execute("SELECT * FROM tbl_musician")
print(cursor.fetchall())

In [ ]:
con.close() # закрыть соединение

In [ ]:
# если нужно получить строки в виде словарей
con = sqlite3.connect('musicians.db3')
con.row_factory = sqlite3.Row
cursor = con.cursor() 
cursor.execute("SELECT * FROM tbl_musician")
print([dict(row) for row in cursor.fetchall()])

In [ ]:
# можно просто перебрать содержимое курсора в цикле for
res = cursor.execute("SELECT * FROM tbl_musician")
for row in res:
    print(dict(row))

In [ ]:
print(res)

Запрос на добавление записи выполняется также в функции ```execute()``` курсора. Посмотреть количество добавленных/измененных записей можно, обратившись к свойству ```cursor.rowcount```.


In [ ]:
cursor.execute("INSERT INTO tbl_musician (musFirstName, musLastName) VALUES ('Barry', 'White')")
con.commit()
print(f"{cursor.rowcount} rows added")

In [ ]:
%sql SELECT * FROM tbl_musician

### ЗАДАНИЕ

Выведите всех музыкантов всех групп в формате:
```
The Beatles are: Lennon, Harrison, McCartney, Starr
The White Stripes are: ...
и т.д.
```

In [ ]:
# ваш код здесь


### ЗАДАНИЕ НА ДОМ

Создайте справочник tbl_occupation с содержимым 'singer' (певец), 'guitarist' (гитарист), 'bassist' (басист) и 'drummer' (ударник). Реализуйте связку многие-ко-многим через таблицу tbl_musician_occupation для вышеозначенных музыкантов и выведите списки всех музыкантов по их специальностям в виде:
```
Singers: Barry White, Jonh Lennon, ...
Guitarists: George Harrison, Jack White, Jimmy Page
...
``` 

Для справки:
```
John Lennon - singer, guitarist
George Harrison - singer, guitarist
Paul McCartney - singer, bassist
Ringo Starr - singer, durmmer
Jack White - singer, guitarist
Meg White - drummer
Robert Plant - singer
Jimmy Page - guitarist
Barry White - singer
```

In [ ]:
# ваш код здесь
